<a href="https://colab.research.google.com/github/jyporse/YOLO/blob/main/Keras_Custom_Data_Medel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/Gdrive')

In [ ]:
import cv2
import numpy as np
import math
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [ ]:
batch_size = 128
num_classes = 0
epochs = 100
CW = 32
CH = 32
CD = 3
model_name = 'fruit_custom.h5'
tflite_model_name = 'fruit_custom.tflite'
COLAB_PATH = '/content/Gdrive/MyDrive/Kobot/YOLO/darknet'
YOLO_IMAGE_PATH =COLAB_PATH + '/custom/'
YOLO_FORMAT_PATH =COLAB_PATH+  '/custom/'
classes = []

train_imgs= []
train_labels = []
test_imgs = []
test_labels = []

sample_test_img = 'sample.test.jpg'
sample_test_label = 0

In [ ]:
def show_sample(img, labels, sample_count = 25):
    grid_cnt = math.ceil(math.ceil(math.sqrt(sample_count)))
    grid_cnt = min(grid_cnt, len(img,), len(labels))

    plt.figure(figsize=(2*grid_cnt, 2*grid_cnt))
    for i in range(sample_count):
        plt.subplot(grid_cnt, grid_cnt, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        image = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
        plt.imshow(image, cmap=plt.cm.gray)
        plt.xlabel(labels[i])
    plt.show()

def download(path):
    try:
        from google.colab import files
        files.download(path)
    except ImportError:
        import os
        print("Eorror Download:", os.path.join(os.getcwd(),path))


def getROI(size, box):
    width_ratio = size[1]
    height_ratio = size[0]
    x = float(box[1]) * width_ratio
    y = float(box[2]) * height_ratio
    w = float(box[3]) * width_ratio   
    h = float(box[4]) * height_ratio

    half_width = w/2.0
    half_height = h/2.0
    startX = int(x-half_width)
    startY = int(y-half_height)
    endX = int(x+half_width)
    endY = int(y+half_height)
    return (startY, endY, startX, endX)

def imShow(path):
    fig = plt.gcf()
    plt.axis("off")
    plt.imshow(cv2.cvtColor(path, cv2.COLOR_BGR2RGB))
    plt.show()


In [ ]:
with open(YOLO_FORMAT_PATH + 'classes.txt' , 'r') as txt:
    for line in txt:
        name = line.replace("\n", "")
        classes.append(name)
        num_classes +=1
    print(classes, num_classes)

In [ ]:
if not os.path.exists('darknet'):
    os.makedirs('darknet')
%cd darknet
!cp -r /content/Gdrive/MyDrive/Kobot/YOLO/darknet/bin/darknet ./darknet
!chmod +x ./darknet
!cp -r /content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom .

## train

In [ ]:
image_cnt = 0
with open(YOLO_FORMAT_PATH + 'train.txt' , 'r') as txt:
    for line in txt:
        
        image_path = line.replace("\n","")
        img = cv2.imread(image_path)
        size = img.shape[:2]
        text_path = image_path[:-4] + '.txt'

        with open(text_path, 'r') as txt:
            for line in txt:
                box = line.split()
                (startY, endY, startX, endX) = getROI(size, box)
                image = cv2.resize(img[startY:endY, startX:endX], (CW,CH), interpolation = cv2.INTER_AREA)
                train_imgs.append(image)
                train_labels.append(int(box[0]))
                image_cnt +=1

train_images = np.array(train_imgs)
train_labels = np.array(train_labels)

train_images =  train_images.astype("float32") /255.0
train_labels = keras.utils.to_categorical(train_labels, num_classes)
print('%d images added' % image_cnt)

## test 

In [ ]:
image_cnt = 0
with open(YOLO_FORMAT_PATH + 'test.txt' , 'r') as txt:
    for line in txt:
        
        image_path = line.replace("\n","")
        img = cv2.imread(image_path)
        size = img.shape[:2]
        text_path = image_path[:-4] + '.txt'

        with open(text_path, 'r') as txt:
            for line in txt:
                box = line.split()
                (startY, endY, startX, endX) = getROI(size, box)
                image = cv2.resize(img[startY:endY, startX:endX], (CW,CH), interpolation = cv2.INTER_AREA)
                test_imgs.append(image)
                test_labels.append(int(box[0]))
                image_cnt +=1

test_images = np.array(test_imgs)
test_labels = np.array(test_labels)

test_images =  test_images.astype("float32") /255.0
test_labels = keras.utils.to_categorical(test_labels, num_classes)
print('%d images added' % image_cnt)

In [ ]:
show_number = len(train_images)
print("Tatal number of Images : %d"  % show_number)

if(show_number >25):
    show_number = 25
show_sample(train_images, 
            [ '%s' % classes[np.argmax(label)] for label in train_labels], show_number)

In [ ]:
show_number = len(test_images)
print("Tatal number of Images : %d"  % show_number)

if(show_number >25):
    show_number = 25
show_sample(test_images, 
            [ '%s' % classes[np.argmax(label)] for label in test_labels], show_number)

## Train a TensorFlow model to classify digit images

In [ ]:
inputShape = (CH, CW, CD)

model = keras.Sequential([
                          keras.layers.Flatten(input_shape = inputShape),

                          keras.layers.Reshape(target_shape = inputShape),
                          keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = tf.nn.relu),
                          keras.layers.Conv2D(filters=64, kernel_size =(3,3), activation=tf.nn.relu),
                          keras.layers.MaxPooling2D(pool_size =(2,2,)),
                          keras.layers.Dropout(0.25),
                          keras.layers.Flatten(input_shape=(28,28)),
                          keras.layers.Dense(128, activation = tf.nn.relu),
                          keras.layers.Dropout(0.5),

                          keras.layers.Dense(num_classes, activation= tf.nn.softmax)
])

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# Save the best model as digits_model.h5
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(model_name, save_best_only = True)

# Define a callback to monitor val_loss
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                 patience =5)

In [ ]:
# Train the model usisng the early stopping callback
history = model.fit(train_images, train_labels,
                    validation_data= (test_images, test_labels),
                    epochs = epochs, batch_size = batch_size,
                    callbacks =[earlyStopping, modelCheckpoint])
                    

In [ ]:
download(model_name)

Evaluate model

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print("Test Accuracy:", test_acc)

fig, loss_ax = plt.subplots()
gix, acc_ax = plt.subplots()

loss_ax.plot(history.history['loss'], 'ro', label = 'train loss')
loss_ax.plot(history.history['val_loss'], 'r:', label= 'val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['accuracy'], 'bp', label = 'train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'b:', label= 'val accuracy')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
predictions = model.predict(test_images)

show_sample(test_images,
            ['%s: %.2f' % (classes[np.argmax(result)], result[np.argmax(result)]*100) for result in predictions], 
            len(predictions))

In [ ]:
seleted_digit = 1

result = predictions[seleted_digit]
result_number = np.argmax(result)
print('%s : %.2f' % (classes[result_number], result[result_number] * 100))

imShow(test_images[seleted_digit])